# SQLite Database Builder
By Kenneth Burchfiel

Released under the MIT license

This code converts 4 separate data sources into one SQLite database. Pandas DataFrames are used as a bridge between those data sources and SQL tables given the ease of use of the to_sql Pandas function. SQLAlchemy was used as the connector because it supports other types of databases (including PostgreSQL) as well.

Because 'if exists == replace' was included in the to_sql functions, this file can be run multiple times without creating duplicate tables/entries in the SQLite database.

In [1]:
import time
start_time = time.time() # Allows the program's runtime to be measured
import sqlite3
import pandas as pd
import os
import sqlalchemy
# from sqlalchemy.orm import Session

# Part 1: Creating the SQLite database

The following code blocks connect to the SQLite database (or create it if it does not already exist)

In [2]:
sqlalchemy_sqlite_engine = sqlalchemy.create_engine('sqlite:///data\\sqlite_database.db') 
# Based on https://docs.sqlalchemy.org/en/13/dialects/sqlite.html#connect-strings
# Creates a database if one does not exist already


This database will store four different tables:
1. A 'flights' table with US passenger traffic data. The source for this data was the T-100 Segment (All Carriers) table from the Bureau of Transportation Statistics (https://www.transtats.bts.gov/Tables.asp?QO_VQ=EEE&QO_anzr=Nv4%FDPn44vr4%FDf6n6v56vp5%FD%FLS14z%FDHE%FDg4nssvp%FM-%FDNyy%FDPn44vr45&QO_fu146_anzr=Nv4%FDPn44vr45).
2. A 'steps' table containing some of my Fitbit steps data 
3. A 'music' table containing information about 64 public-domain music clips. I found music available under a CC0 (public domain) license at https://freepd.com/. Since I like electronic music, I downloaded all of the tracks within the electronic music section (https://freepd.com/electronic.php). Of these tracks, ‘Fireworks’ by Alexander Nakarada was my favorite. I then went into Audacity and created 64 short clips of this song.
4. A 'photos' table containing information about public-domain photos that I sourced from NASA and Flickr. Most Flickr photos were created by the US Government, but I also found public-domain images from other Flickr users as well. 

All of this data is free to use.

## 1a: Creating the 'flights' Table

In [3]:
df_flights = pd.read_csv('data\\routes_planes_coordinates.csv')
df_flights.drop('Unnamed: 0', axis=1, inplace=True)
df_flights.drop('Unnamed: 50', axis=1, inplace=True)
df_flights


,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.640,-73.779
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.980,-95.340,SAV,32.127,-81.202
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787
4,0.0,1.0,12500.0,50.0,34.0,0.0,0.0,133.0,49.0,29.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482268,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,...,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.060,BOS,42.364,-71.005
482269,1188.0,594.0,1069200.0,5346.0,3573.0,0.0,0.0,91.0,27799.0,21740.0,...,1,"Piston, 2-Engine",1,Passenger Configuration,BOS,42.364,-71.005,ACK,41.253,-70.060
482270,1216.0,608.0,1094400.0,5472.0,3827.0,0.0,0.0,91.0,28454.0,22253.0,...,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.060,BOS,42.364,-71.005
482271,1258.0,629.0,1132200.0,5661.0,4056.0,0.0,0.0,91.0,29437.0,23021.0,...,1,"Piston, 2-Engine",1,Passenger Configuration,BOS,42.364,-71.005,ACK,41.253,-70.060


The to_sql function easily imports these 482,273 rows and 62 columns into the SQLite database. If that column already exists, it will be replaced by the new table (due to the 'if_exists = 'replace'' argument).

In [4]:
df_flights.to_sql('flights', con = sqlalchemy_sqlite_engine, if_exists = 'replace',
index_label = 'index')
# Source: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
# I learned about this method from 'Tennessee Leeuwenburg'
# (https://stackoverflow.com/a/28802613/13097194)
# Giving the index column a name makes importing this data somewhat easier.

The following code block confirms that this table can in turn be read from the database back into a Pandas DataFrame.

In [5]:
df_flights_from_db = pd.read_sql("Select * from flights", index_col='index', con = sqlalchemy_sqlite_engine)
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html
df_flights_from_db

,DEPARTURES_SCHEDULED,DEPARTURES_PERFORMED,PAYLOAD,SEATS,PASSENGERS,FREIGHT,MAIL,DISTANCE,RAMP_TO_RAMP,AIR_TIME,...,Code_y,Plane_Group_Text,Code,Plane_Config_Text,origin_iata_code,origin_lat,origin_lon,destination_iata_code,destination_lat,destination_lon
index,,,,,,,,,,,,,,,,,,,,,
0,0.0,1.0,21502.0,76.0,3.0,0.0,0.0,901.0,170.0,140.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,FLL,26.072,-80.153
1,0.0,3.0,64506.0,228.0,75.0,0.0,0.0,228.0,219.0,140.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IAD,38.944,-77.456,JFK,40.640,-73.779
2,0.0,1.0,21502.0,76.0,64.0,0.0,0.0,851.0,144.0,114.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IAH,29.980,-95.340,SAV,32.127,-81.202
3,0.0,1.0,21502.0,76.0,55.0,0.0,0.0,122.0,58.0,31.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,ILM,34.271,-77.903,RDU,35.877,-78.787
4,0.0,1.0,12500.0,50.0,34.0,0.0,0.0,133.0,49.0,29.0,...,6,"Jet, 2-Engine",1,Passenger Configuration,IND,39.717,-86.294,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
482268,1166.0,583.0,1049400.0,5247.0,3646.0,0.0,0.0,91.0,27284.0,21338.0,...,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.060,BOS,42.364,-71.005
482269,1188.0,594.0,1069200.0,5346.0,3573.0,0.0,0.0,91.0,27799.0,21740.0,...,1,"Piston, 2-Engine",1,Passenger Configuration,BOS,42.364,-71.005,ACK,41.253,-70.060
482270,1216.0,608.0,1094400.0,5472.0,3827.0,0.0,0.0,91.0,28454.0,22253.0,...,1,"Piston, 2-Engine",1,Passenger Configuration,ACK,41.253,-70.060,BOS,42.364,-71.005


## 1b: Creating 'photos' Table

To create the 'photos' and 'music' tables, I could have stored the original photo and music data within the table in binary format. However, a Stack Overflow discussion located at https://stackoverflow.com/questions/154707/what-is-the-best-way-to-store-media-files-on-a-database) led me to conclude that keeping the files within a separate storage option (such as a Google Cloud Storage bucket) made more sense. Therefore, both the 'photos' and 'music' tables will store file information rather than the files themselves.

The code block below converts information about the photos in the data\public_domain_photos_for_db folder into a DataFrame. The google_cloud_storage_bucket, cloud_storage_folder, and gcs_url_root strings are used to construct the Google Cloud Storage URL where each file could be located. (I subsequently deleted this storage bucket in order to save storage costs, so the URLs will no longer work. To replicate this approach, you will need to create your own storage bucket and upload the files there.)

In [6]:
google_cloud_storage_bucket = 'kb_sample_database' # The name (within the 
# Google URL) where image data will be stored
cloud_storage_folder = 'pub_domain_images'
gcs_url_root = 'https://storage.googleapis.com'
photo_location = "data\\public_domain_photos_for_db\\" 
photo_list = os.listdir(photo_location)
print(type(photo_list))
# print(photo_list)
photo_dict_list = []
for photo_path in photo_list:
    full_path = photo_location + photo_path        
    stat_file = os.stat(full_path)
    gcs_url = (gcs_url_root+'/'+google_cloud_storage_bucket+'/'
    + cloud_storage_folder + '/' + photo_path)
    # Public Google Cloud Storage URL
    # print(type(test_photo)) # The class is '_io.BufferedReader'
    # print(type(time.ctime(stat_file.st_ctime))) # These are stored as strings
    photo_dict_list.append({'file_name': photo_path, 'size': stat_file.st_size, 'created_date': time.ctime(stat_file.st_ctime), 'modified_date': time.ctime(stat_file.st_mtime), 'gcs_url': gcs_url}) 
df_photos = pd.DataFrame(photo_dict_list)
df_photos

<class 'list'>


,file_name,size,created_date,modified_date,gcs_url
0,10086848403_b33a695758_o.jpg,1027202,Sat Nov 13 11:34:49 2021,Mon Sep 13 20:44:36 2021,https://storage.googleapis.com/kb_sample_datab...
1,10171744985_76f8973d6b_o.jpg,160300,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:42:14 2021,https://storage.googleapis.com/kb_sample_datab...
2,10822424126_91be9abb6d_o.jpg,402898,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:42:25 2021,https://storage.googleapis.com/kb_sample_datab...
3,10843413524_82caa8b0f8_o.jpg,7405423,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:32:56 2021,https://storage.googleapis.com/kb_sample_datab...
4,11309341065_6fcfbee752_o.jpg,466609,Sat Nov 13 11:34:49 2021,Mon Sep 13 20:38:00 2021,https://storage.googleapis.com/kb_sample_datab...
...,...,...,...,...,...
128,S69-34316~orig.jpg,1339165,Sat Nov 13 11:34:51 2021,Mon Sep 13 01:45:54 2021,https://storage.googleapis.com/kb_sample_datab...
129,S71-41357~orig.jpg,2103357,Sat Nov 13 11:34:51 2021,Mon Sep 13 20:48:45 2021,https://storage.googleapis.com/kb_sample_datab...
130,S71-41759~orig.jpg,1470210,Sat Nov 13 11:34:51 2021,Mon Sep 13 20:07:29 2021,https://storage.googleapis.com/kb_sample_datab...
131,sts061-s-104~orig.jpg,2055089,Sat Nov 13 11:34:51 2021,Mon Sep 13 20:58:49 2021,https://storage.googleapis.com/kb_sample_datab...


As with the 'flights' table, to_sql will be used to import the photos DataFrame into the SQLite database, and read_sql will be used to ensure that the information was imported successfully.

In [7]:
df_photos.to_sql('photos', con = sqlalchemy_sqlite_engine, if_exists = 'replace', index_label = 'index')

In [8]:
df_photos_from_db = pd.read_sql("Select * from photos", index_col='index', con = sqlalchemy_sqlite_engine)
df_photos_from_db

,file_name,size,created_date,modified_date,gcs_url
index,,,,,
0,10086848403_b33a695758_o.jpg,1027202,Sat Nov 13 11:34:49 2021,Mon Sep 13 20:44:36 2021,https://storage.googleapis.com/kb_sample_datab...
1,10171744985_76f8973d6b_o.jpg,160300,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:42:14 2021,https://storage.googleapis.com/kb_sample_datab...
2,10822424126_91be9abb6d_o.jpg,402898,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:42:25 2021,https://storage.googleapis.com/kb_sample_datab...
3,10843413524_82caa8b0f8_o.jpg,7405423,Sat Nov 13 11:34:49 2021,Mon Sep 13 01:32:56 2021,https://storage.googleapis.com/kb_sample_datab...
4,11309341065_6fcfbee752_o.jpg,466609,Sat Nov 13 11:34:49 2021,Mon Sep 13 20:38:00 2021,https://storage.googleapis.com/kb_sample_datab...
...,...,...,...,...,...
128,S69-34316~orig.jpg,1339165,Sat Nov 13 11:34:51 2021,Mon Sep 13 01:45:54 2021,https://storage.googleapis.com/kb_sample_datab...
129,S71-41357~orig.jpg,2103357,Sat Nov 13 11:34:51 2021,Mon Sep 13 20:48:45 2021,https://storage.googleapis.com/kb_sample_datab...
130,S71-41759~orig.jpg,1470210,Sat Nov 13 11:34:51 2021,Mon Sep 13 20:07:29 2021,https://storage.googleapis.com/kb_sample_datab...


## 1c: Creating the 'steps' Table

I obtained the steps data from my online Fitbit account as a series of .json files, then consolidated these files into a single .json file. The below code block converts this .json file into a DataFrame.

In [9]:
df_steps = pd.io.json.read_json('data\\combined_steps_file.json') # Took 34.6 seconds to run
# See https://pandas.pydata.org/docs/reference/api/pandas.io.json.read_json.html

In [10]:
df_steps

,dateTime,value
0,2020-01-20 19:05:00,0
1,2020-01-20 19:38:00,0
2,2020-01-20 19:39:00,0
3,2020-01-20 19:40:00,0
4,2020-01-20 19:41:00,61
...,...,...
529251,2021-09-16 18:26:00,0
529252,2021-09-16 18:27:00,0
529253,2021-09-16 18:28:00,0
529254,2021-09-16 18:29:00,0


I'll once again use to_sql and read_sql to import this DataFrame into my SQLite database, then read it back into a DataFrame.

In [11]:
df_steps.to_sql('steps', con = sqlalchemy_sqlite_engine, if_exists = 'replace',
index_label = 'index')

In [12]:
df_steps_from_db = pd.read_sql("Select * from steps", index_col='index', con = sqlalchemy_sqlite_engine)
df_steps_from_db

,dateTime,value
index,,
0,2020-01-20 19:05:00.000000,0
1,2020-01-20 19:38:00.000000,0
2,2020-01-20 19:39:00.000000,0
3,2020-01-20 19:40:00.000000,0
4,2020-01-20 19:41:00.000000,61
...,...,...
529251,2021-09-16 18:26:00.000000,0
529252,2021-09-16 18:27:00.000000,0
529253,2021-09-16 18:28:00.000000,0


# 1d: Creating 'music' Table

The 'music' table is created using the same method that I used to construct the 'photos' table.

In [13]:
google_cloud_storage_bucket = 'kb_sample_database' # The name (within the 
# Google URL) where music data will be stored
cloud_storage_folder = 'pub_domain_music'
gcs_url_root = 'https://storage.googleapis.com'
music_location = "data\\public_domain_music\\" 
music_list = os.listdir(music_location)

music_dict_list = []
for music_path in music_list:
    full_path = music_location + music_path        
    stat_file = os.stat(full_path)
    gcs_url = (gcs_url_root+'/'+google_cloud_storage_bucket+'/'
    + cloud_storage_folder + '/' + music_path)
    # Public Google Cloud Storage URL
    music_dict_list.append({'music_file_name': music_path, 'music_size': stat_file.st_size, 'music_created_date': time.ctime(stat_file.st_ctime), 'music_modified_date': time.ctime(stat_file.st_mtime), 'music_gcs_url': gcs_url}) 
df_music = pd.DataFrame(music_dict_list)
df_music

,music_file_name,music_size,music_created_date,music_modified_date,music_gcs_url
0,sample_0.mp3,94391,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:32:27 2021,https://storage.googleapis.com/kb_sample_datab...
1,sample_1.mp3,114244,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:40:16 2021,https://storage.googleapis.com/kb_sample_datab...
2,sample_10.mp3,73493,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:14 2021,https://storage.googleapis.com/kb_sample_datab...
3,sample_11.mp3,94391,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:18 2021,https://storage.googleapis.com/kb_sample_datab...
4,sample_12.mp3,64088,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:22 2021,https://storage.googleapis.com/kb_sample_datab...
...,...,...,...,...,...
59,sample_62.mp3,73493,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:38:15 2021,https://storage.googleapis.com/kb_sample_datab...
60,sample_63.mp3,103795,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:38:19 2021,https://storage.googleapis.com/kb_sample_datab...
61,sample_7.mp3,64088,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:03 2021,https://storage.googleapis.com/kb_sample_datab...
62,sample_8.mp3,73493,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:06 2021,https://storage.googleapis.com/kb_sample_datab...


to_sql and read_sql are used once again to import data into SQLite and ensure it can be read back into a DataFrame.

In [14]:
df_music.to_sql('music', con = sqlalchemy_sqlite_engine, if_exists = 'replace', index_label = 'index')

In [15]:
df_music_from_db = pd.read_sql("Select * from music", index_col='index', con = sqlalchemy_sqlite_engine)
df_music_from_db

,music_file_name,music_size,music_created_date,music_modified_date,music_gcs_url
index,,,,,
0,sample_0.mp3,94391,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:32:27 2021,https://storage.googleapis.com/kb_sample_datab...
1,sample_1.mp3,114244,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:40:16 2021,https://storage.googleapis.com/kb_sample_datab...
2,sample_10.mp3,73493,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:14 2021,https://storage.googleapis.com/kb_sample_datab...
3,sample_11.mp3,94391,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:18 2021,https://storage.googleapis.com/kb_sample_datab...
4,sample_12.mp3,64088,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:22 2021,https://storage.googleapis.com/kb_sample_datab...
...,...,...,...,...,...
59,sample_62.mp3,73493,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:38:15 2021,https://storage.googleapis.com/kb_sample_datab...
60,sample_63.mp3,103795,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:38:19 2021,https://storage.googleapis.com/kb_sample_datab...
61,sample_7.mp3,64088,Sat Nov 13 11:34:51 2021,Thu Sep 16 15:34:03 2021,https://storage.googleapis.com/kb_sample_datab...


Now that the SQLite database has been created, I can import its tables into various online databases (Amazon Web Services, Google Cloud Platform, Azure, etc.) within the database_uploader.ipynb file.

In [16]:
end_time = time.time()
run_time = end_time - start_time
run_minutes = run_time // 60
run_seconds = run_time % 60
print("Completed run at",time.ctime(end_time),"(local time)")
print("Total run time:",'{:.2f}'.format(run_time),"second(s) ("+str(run_minutes),"minute(s) and",'{:.2f}'.format(run_seconds),"second(s))") # Only meaningful when the program is run nonstop from start to finish

Completed run at Sat Nov 13 12:23:07 2021 (local time)
Total run time: 69.82 second(s) (1.0 minute(s) and 9.82 second(s))
